# Lista 3 - Classificador

Para resolução da lista utilize os slides do material de pré-processamento e classificação e os códigos de pré-processamentoe classificação.

## **Exercício**:
Utilizando o banco de dados do IMDB (utilizado na aula de classificação).

a) Realize o pré-processamento do dado utilizando o seguinte pré-processamento:

* Tokenizar as strings utilizando nltk
* Passar as palavras para minúsculo
* Remover acentos
* Remover números
* Remover pontos
* Realizar lemmantization

b) Treine um modelo de random forest para prever se o texto é uma review positiva ou negativa. Treine um modelo para cada uma das representações abaixo:

1. One hot encoding
2. Bag-of-words (com a frequencia das palavras)
3. TF-IDF

c) Calcule as métricas dos modelos de b. Qual deles você considera o melhor e porque?

d) Quais palavras (features) foram as mais importantes do modelo selecionado?



In [39]:
import pandas as pd
        
imdb_reviews_pt_br = pd.read_csv('../../../Datasets/imdb-reviews-pt-br.csv', sep = ',')
imdb_reviews_pt_br

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg
...,...,...,...,...
49454,49456,"Seeing as the vote average was pretty low, and...","Como a média de votos era muito baixa, e o fat...",pos
49455,49457,"The plot had some wretched, unbelievable twist...",O enredo teve algumas reviravoltas infelizes e...,pos
49456,49458,I am amazed at how this movieand most others h...,Estou espantado com a forma como este filme e ...,pos
49457,49459,A Christmas Together actually came before my t...,A Christmas Together realmente veio antes do m...,pos


In [40]:
imdb_reviews_pt_br.isna().sum()

id           0
text_en      0
text_pt      0
sentiment    0
dtype: int64

In [41]:
imdb_reviews_pt_br.dtypes

id            int64
text_en      object
text_pt      object
sentiment    object
dtype: object

In [42]:
print(imdb_reviews_pt_br['text_pt'].iloc[0])

Mais uma vez, o Sr. Costner arrumou um filme por muito mais tempo do que o necessário. Além das terríveis seqüências de resgate no mar, das quais há muito poucas, eu simplesmente não me importei com nenhum dos personagens. A maioria de nós tem fantasmas no armário, e o personagem Costers é realizado logo no início, e depois esquecido até muito mais tarde, quando eu não me importava. O personagem com o qual deveríamos nos importar é muito arrogante e superconfiante, Ashton Kutcher. O problema é que ele sai como um garoto que pensa que é melhor do que qualquer outra pessoa ao seu redor e não mostra sinais de um armário desordenado. Seu único obstáculo parece estar vencendo Costner. Finalmente, quando estamos bem além do meio do caminho, Costner nos conta sobre os fantasmas dos Kutchers. Somos informados de por que Kutcher é levado a ser o melhor sem pressentimentos ou presságios anteriores. Nenhuma mágica aqui, era tudo que eu podia fazer para não desligar uma hora.


In [43]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import unidecode
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lucia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lucia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### a) Realize o pré-processamento do dado utilizando o seguinte pré-processamento:

In [44]:
# Pré-processamento
def preprocess_text(text):
    text = word_tokenize(text) # Tokenizar as strings utilizando nltk
    text = [word.lower() for word in text] # Passar as palavras para minúsculo
    text = [unidecode.unidecode(word) for word in text] # Remover acentos   
    text = [word for word in text if not word.isdigit()] # Remover números
    text = [word for word in text if word.isalpha()] # Remover pontos
    lemmatizer = WordNetLemmatizer() # Realizar lemmantization  
    text = [lemmatizer.lemmatize(word) for word in text] # Realizar lemmantization
    return ' '.join(text) # Juntar tokens de volta em strings

imdb_reviews_pt_br['text_pt'] = imdb_reviews_pt_br['text_pt'].apply(preprocess_text)

### b) Treine um modelo de random forest para prever se o texto é uma review positiva ou negativa. Treine um modelo para cada uma das representações abaixo:

#### 1. One hot encoding

In [59]:
# One hot encoding
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# Convertendo as labels para valores binários
y = imdb_reviews_pt_br['sentiment'].apply(lambda x: 1 if x == 'pos' else 0)

In [67]:
def train_and_get_feature_importances(vectorizer, X_text, y, vectorizer_name):
    
    X = vectorizer.fit_transform(X_text)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_classifier.fit(X_train, y_train)

    y_pred = rf_classifier.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    features = vectorizer.get_feature_names_out()
    importances = rf_classifier.feature_importances_
    importances_df = pd.DataFrame(importances, index=features, columns=['importance']).sort_values('importance', ascending=False)


    return importances_df, report , accuracy
    

In [68]:
# Usando CountVectorizer para criar uma matriz binária
count_vectorizer = CountVectorizer(binary=True)
importances_one, report_one, accuracy_one = train_and_get_feature_importances(count_vectorizer, imdb_reviews_pt_br['text_pt'], y, 'One hot encoding')



#### 2. Bag-of-words (com a frequencia das palavras)    

In [69]:
# Bag-of-words
count_vectorizer = CountVectorizer()
importances_bag, report_bag, accuracy_bag = train_and_get_feature_importances(count_vectorizer, imdb_reviews_pt_br['text_pt'], y, 'Bag-of-words')   

#### 3. TF-IDF

In [70]:
# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
importances_tfidf, report_tfidf, accuracy_tfidf = train_and_get_feature_importances(tfidf_vectorizer, imdb_reviews_pt_br['text_pt'], y, 'TF-IDF')

### c) Calcule as métricas dos modelos de b. Qual deles você considera o melhor e porque?

In [71]:
print('One hot encoding')
print('Accuracy:', accuracy_one)
print(report_one)

print('Bag-of-words')
print('Accuracy:', accuracy_bag)
print(report_bag)

print('TF-IDF')
print('Accuracy:', accuracy_tfidf)  
print(report_tfidf)



One hot encoding
Accuracy: 0.8397695107157299
              precision    recall  f1-score   support

           0       0.84      0.85      0.84      5038
           1       0.84      0.83      0.84      4854

    accuracy                           0.84      9892
   macro avg       0.84      0.84      0.84      9892
weighted avg       0.84      0.84      0.84      9892

Bag-of-words
Accuracy: 0.8411847957945815
              precision    recall  f1-score   support

           0       0.84      0.84      0.84      5038
           1       0.84      0.84      0.84      4854

    accuracy                           0.84      9892
   macro avg       0.84      0.84      0.84      9892
weighted avg       0.84      0.84      0.84      9892

TF-IDF
Accuracy: 0.8283461382935705
              precision    recall  f1-score   support

           0       0.83      0.84      0.83      5038
           1       0.83      0.82      0.82      4854

    accuracy                           0.83      9892
   m

### d) Quais palavras (features) foram as mais importantes do modelo selecionado?

In [72]:
print('Features mais importantes:')
print('    One hot encoding')
print(importances_one.head(10)) 

print('    Bag-of-words')
print(importances_bag.head(10))

print('    TF-IDF')
print(importances_tfidf.head(10))


Features mais importantes:
    One hot encoding
           importance
ruim         0.008630
pior         0.008494
horrivel     0.005430
excelente    0.004595
otimo        0.004269
piores       0.004041
nada         0.003502
terrivel     0.003449
nem          0.003244
lixo         0.002624
    Bag-of-words
           importance
ruim         0.010364
pior         0.009323
horrivel     0.005484
nao          0.005412
excelente    0.004417
piores       0.003980
otimo        0.003918
terrivel     0.003740
nada         0.003124
lixo         0.002871
    TF-IDF
           importance
ruim         0.009787
pior         0.007751
nao          0.007358
horrivel     0.005259
terrivel     0.004726
excelente    0.004414
otimo        0.003990
piores       0.003325
nada         0.003150
nem          0.003059
